<a href="https://colab.research.google.com/github/ksr1710/Ejercicios-basicos-html-y-js/blob/main/PQRS_TDC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🤖 **Azure OpenAI API para servicio (Encuestas y E-mails)**

### ⚙️ **Instalación Herramienta** ⚙️

In [ ]:
%%capture
from google.cloud import storage

path_to_private_key = './analitica-poc-gcp-691e6bb207d0.json'
client = storage.Client.from_service_account_json(json_credentials_path=path_to_private_key)
bucket = storage.Bucket(client, 'nlp_servicio_encuestas_mails_colab')
blob = bucket.blob('nlp_servicio_encuestas_mails_colab_v1.zip')
blob.download_to_filename('nlp_servicio_encuestas_mails_colab_v1.zip')

!rm -rf sample_data; unzip -P "Okic,\4m-3[EE2+/a?GY3;VT)#J5z#I0" nlp_servicio_encuestas_mails_colab_v1.zip; rm nlp_servicio_encuestas_mails_colab_v1.zip; pip3 install -r requirements.txt; cd gpt4service; python setup_cython.py build_ext --inplace; rm -rf gpt4service.c gpt4service.pyx

### ⚙️ **GPT config** ⚙️

##### *Cargar config*

In [ ]:
#Note: The openai-python library support for Azure OpenAI is in preview.
import openai
import json
from gpt4service import utils
from gpt4service import GPTService

# Opening JSON file
f = open('config_gpt.json')

# returns JSON object as
# a dictionary
config_gpt = json.load(f)

# cargar nombre del modelo - Recomendado GPT 3.5 turbo
gpt_engine = config_gpt["gpt_engine"]

# cargar prompts
system_message_prompt = config_gpt["system_message_prompt"]
prompt_gpt_comments = config_gpt["prompt_gpt_comments"]
prompt_gpt_email = config_gpt["prompt_gpt_email"]

# objeto de extracción
servicer = GPTService(prompt_comentarios=prompt_gpt_comments,
                    prompt_emails=prompt_gpt_email,
                    system_message_prompt=system_message_prompt
                    )
# Closing file
f.close()

##### *Cambiar prompts config*
> IMPORTANTE!
Usar solo en caso de que se desee modificar los prompts de extracción

In [ ]:
import json

system_message_prompt = "Eres un asistente de inteligencia artificial que ayuda en el departamento de atención al cliente del banco Davivienda a diagnosticar la experiencia de los clientes  de tarjeta de credito. Los clientes son los usuarios de las tarjetas de credito  que se comunican con la el banco y se deja una solicitud de servicio.  Para facilitar tu tarea de analisis, debes saber lo siguiente: La respuesta del cliente se da luego que el banco le notifique si su solicitud se resolvio de manera exitosa o no. y pueden serguir inconformes con la solicitud a su tarjeta de crecito Tienes que dar tus respuestas en la estructura de un archivo json"


prompt_gpt_comments = """Responde en la estructura de un archivo json:

{
    "score_satisfaccion": "valor de 1 a 10, donde 10 es muy satisfecho y 1 muy insatisfecho",
    "razon_score": "explica de manera breve por qué asignaste el score_satisfacción que asignaste",
    "motivo_comentario": "responde con el motivo por el cual el cliente hace el comentario, por ejemplo: buena o mala atención, aplicación móvil, altas tasas de interés, etc. Responde de forma corta y puntual, máximo 5 palabras.",
    "accion_sugerida": "si el comentario tiende a ser negativo, responde este campo con una sugerencia de acción. Si el comentario no es muy extenso puedes responder este campo con 'NaN'",
    "pqrsf": "categoriza el comentario en petición, queja, reclamo, sugerencia o felicitación. Si no es posible categorizar responde 'NaN'",
    "fidelizacion": "se iría del banco: urgente, neutro, fiel",
    "motivo_comentario": "", // [RESPUESTA OBLIGATORIA] Responde con el motivo por el cual el cliente inicia la conversación (máximo 3 palabras).
    "solicitud_realizada": "", // [RESPUESTA OBLIGATORIA] Solicitud realizada por el cliente. Separar múltiples solicitudes con "_".
    //NOTA: En las siguientes columnas es necesario clasificar el contenido del correo en ciertas categorías fijas. En principio las categorías no
    son excluyentes.
    "probabilidad_Entrega_de_la_tarjeta": "", // [RESPUESTA OBLIGATORIA] Probabilidad de que la categoría sea "Entrega de tarjeta".
    Debe ser un valor entre 0 y 1.
    "probabilidad_Facturación_incorrecta": "", // [RESPUESTA OBLIGATORIA] Probabilidad de que la categoría sea "Facturación_incorrecta".
    Debe ser un valor entre 0 y 1.
    "probabilidad_Cargos_no_reconocidoso": "", // [RESPUESTA OBLIGATORIA] Probabilidad de que la categoría sea "Cargos no reconocidoso".
    Debe ser un valor entre 0 y 1.
    "probabilidad_Tasas_de_interés_y_comisioneso": "", // [RESPUESTA OBLIGATORIA] Probabilidad de que la categoría sea "Tasas de interés  y comisiones".
    Debe ser un valor entre 0 y 1.
    "probabilidad_Cancelación_no_solicitadan": "", // [RESPUESTA OBLIGATORIA] Probabilidad de que la categoría sea "Cancelacion del producto".
    Debe ser un valor entre 0 y 1.
    "probabilidad_Problemas_con_el_límite_de_créditos": "", // [RESPUESTA OBLIGATORIA] Probabilidad de que la categoría sea "problemas con limite de credito".
    Debe ser un valor entre 0 y 1.
    "probabilidad_Bloqueo_de_la_tarjeta": "", // [RESPUESTA OBLIGATORIA] Probabilidad de que la categoría sea "bloqueos".
    Debe ser un valor entre 0 y 1.
    "probabilidad_Demoras_o_errores_en_el_proceso_de_pagos": "", // [RESPUESTA OBLIGATORIA] Probabilidad de que la categoría sea "Demoras o errores en el proceso de pagos".
    Debe ser un valor entre 0 y 1.
    "probabilidad_vinculación": "" // [RESPUESTA OBLIGATORIA] Probabilidad de que el motivo no sea ni "problemas en el momento de adquirir su tarjeta por primer vez" ni


}

"""


prompt_gpt_email = """Responde en la estructura de un archivo json:

{
    score_satisfaccion: valor de 1 a 10, donde 10 es muy satisfecho y 1 muy insatisfecho
    razon_score: explica de manera breve por qué asignaste el score_satisfacción que asignaste
    motivo_correo: responde con el motivo por el cual el cliente inicia la conversación, por ejemplo: solicitud de producto, de algún documento, de algún saldo, etc. Responde breve, máximo 5 palabras
    solicitud_realizada: si el cliente solicitó algo, respóndelo aquí. Si son varias solicitudes, escribe todas aquí
    fecha_solicitud_realizada: si el cliente solicitó algo, responde la fecha de la solicitud en formato DD/MM/YYYY. Si son varias solicitudes, escribe todas las fechas aquí
}

"""

data = {}
data['gpt_engine'] = gpt_engine
data['system_message_prompt'] = system_message_prompt
data['prompt_gpt_comments'] = prompt_gpt_comments
data['prompt_gpt_email'] = prompt_gpt_email

with open('config_gpt.json', 'w') as file:
    json.dump(data, file, indent=4)

### 💭 **Comentarios encuestas**

In [ ]:
import pandas as pd
import os

# Read Txt file into DataFrame
#path_encuestas = 'data/encuestas'
#encuestas_file = 'Prom-3.txt'
#df = pd.read_table(os.path.join(path_encuestas, encuestas_file))

# Read TSV file into DataFrame
path_encuestas = 'data/encuestas'
encuestas_file = 'xx.tsv'
df = pd.read_table(os.path.join(path_encuestas, encuestas_file))

columnas_esperadas = ['nro_identificacion', 'nivel_recomendacion', 'voz_cliente', 'voz_recomendacion', 'voz_cliente_inconforme']

# Revisar columnas del df
if columnas_esperadas == list(df.columns):
    print("Columnas esperadas correctas")
else:
    if 'voz_recomendacion' not in list(df.columns):
        df['voz_recomendacion'] = None
    if 'voz_cliente_inconforme' not in list(df.columns):
        df['voz_cliente_inconforme'] = None
    print("Las columnas del dataframe no son las esperadas. Las siguientes funciones podrían no funcionar!!!")
df

Columnas esperadas correctas


,nro_identificacion,nivel_recomendacion,voz_cliente,voz_recomendacion,voz_cliente_inconforme
0,1-26979973647,10,Si el canal de pagos por Redeban cuando se usa...,NaN,NaN
1,1-26981441238,10,recibi la respuesta,NaN,NaN
2,1-27007405641,9,buena,NaN,NaN
3,1-27331170990,10,satisfecho,NaN,NaN
4,1-27856568198,7,"Fue demorado llamar a todos, no me daban soluc...",NaN,NaN
...,...,...,...,...,...
245,1-29085658166,10,Muy buena la atencion,NaN,NaN
246,1-29414318008,9,ME PARECIO BIEN,NaN,NaN
247,1-27386492848,10,Aunque la controversia fue por un monto muy pe...,NaN,NaN
248,1-27709485106,10,deben ser mas rapidas las respuestas,NaN,NaN


In [ ]:
df_detractores = df[df['nivel_recomendacion'] < 7].reset_index(drop=True)

for index, row in df_detractores.iterrows():
    texto = f"Comentario: {row['voz_cliente']}\nRecomendacion: {row['voz_recomendacion']}\nComentario Inconforme: {row['voz_cliente_inconforme']}\n"
    texto_corregido = texto.encode('latin-1', errors='replace').decode('latin-1', errors='replace')

print(f"{texto_corregido}")

Comentario: Favor revisar que esta pasando con los pagos a la cartera en dolares que se hacen por la pagina de Davivienda porque en dos oportunidades me los han abonado a la cartera en pesos. Es un desgaste y perdida de tiempo.
Recomendacion: nan
Comentario Inconforme: nan



#### **GPT respuesta**

##### **Single**

In [ ]:
# prueba de consulta a comentarios
print(f"{texto_corregido}")
servicer.gpt_comments_query(texto_corregido)

Comentario: Favor revisar que esta pasando con los pagos a la cartera en dolares que se hacen por la pagina de Davivienda porque en dos oportunidades me los han abonado a la cartera en pesos. Es un desgaste y perdida de tiempo.
Recomendacion: nan
Comentario Inconforme: nan



{'score_satisfaccion': 4,
 'razon_score': 'El cliente ha tenido problemas con la cartera en dólares',
 'motivo_comentario': 'Problemas con pagos en cartera en dólares',
 'accion_sugerida': 'Revisar y solucionar el problema con la cartera en dólares',
 'pqrsf': 'Queja',
 'fidelizacion': 'Urgente',
 'solicitud_realizada': 'Revisión de pagos en cartera en dólares',
 'probabilidad_Entrega_de_la_tarjeta': 0,
 'probabilidad_Facturación_incorrecta': 0,
 'probabilidad_Cargos_no_reconocidoso': 0,
 'probabilidad_Tasas_de_interés_y_comisioneso': 0,
 'probabilidad_Cancelación_no_solicitadan': 0,
 'probabilidad_Problemas_con_el_límite_de_créditos': 0,
 'probabilidad_Bloqueo_de_la_tarjeta': 0,
 'probabilidad_Demoras_o_errores_en_el_proceso_de_pagos': 1,
 'probabilidad_vinculación': 0,
 'cost': 0.0027}

##### **Batch comentarios**

In [ ]:
#df_loop = df[3:5].copy() # use para procesar parte del df
df_loop = df.copy() # use para procesar todo el df

df_final_encs = servicer.get_comments_df(df_loop)
df_final_encs

100%|██████████| 250/250 [1:07:40<00:00, 16.24s/it]


,nro_identificacion,nivel_recomendacion,voz_cliente,voz_recomendacion,voz_cliente_inconforme,score_satisfaccion,razon_score,motivo_comentario,accion_sugerida,pqrsf,...,probabilidad_Facturación_incorrecta,probabilidad_Cargos_no_reconocidoso,probabilidad_Tasas_de_interés_y_comisioneso,probabilidad_Cancelación_no_solicitadan,probabilidad_Problemas_con_el_límite_de_créditos,probabilidad_Bloqueo_de_la_tarjeta,probabilidad_Demoras_o_errores_en_el_proceso_de_pagos,probabilidad_vinculación,cost,proc_time
0,1-26979973647,10.0,Si el canal de pagos por Redeban cuando se usa...,nan,nan,8.0,El cliente sugiere una mejora en el canal de p...,Canal de pagos,Configurar el sistema para aceptar solo débito...,Sugerencia,...,0,0,0,0,0,0,0,0.2,0.0028,8.05
1,1-26981441238,10.0,recibi la respuesta,nan,nan,7.0,El cliente recibió la respuesta,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0.2,0.0026,5.64
2,1-27007405641,9.0,buena,nan,nan,8.0,El cliente calificó su experiencia como buena,,NaN,felicitación,...,0,0,0,0,0,0,0,0.2,0.0026,6.87
3,1-27331170990,10.0,satisfecho,nan,nan,10.0,El cliente se encuentra satisfecho con el serv...,,NaN,Felicitación,...,0,0,0,0,0,0,0,0,0.0026,7.59
4,1-27856568198,7.0,"Fue demorado llamar a todos, no me daban soluc...",nan,nan,2.0,El cliente se muestra insatisfecho con el serv...,Atención al cliente,Mejorar los tiempos de respuesta y atención al...,Queja,...,0,0,0,0,0,0,0,0.2,0.0027,6.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,1-29085658166,10.0,Muy buena la atencion,nan,nan,10.0,El cliente calificó la atención como muy buena,Atención,NaN,Felicitación,...,0,0,0,0,0,0,0,0.1,0.0026,6.15
246,1-29414318008,9.0,ME PARECIO BIEN,nan,nan,8.0,El cliente se muestra satisfecho con el servic...,,NaN,NaN,...,0,0,0,0,0,0,0,0.2,0.0026,7.16
247,1-27386492848,10.0,Aunque la controversia fue por un monto muy pe...,nan,nan,10.0,El cliente está muy satisfecho con el servicio...,Controversia resuelta,NaN,Felicitación,...,0,0,0,0,0,0,0,0,0.0027,5.97
248,1-27709485106,10.0,deben ser mas rapidas las respuestas,nan,nan,4.0,El cliente indica que las respuestas deben ser...,,Mejorar tiempos de respuesta,Sugerencia,...,0,0,0,0,0,0,0,0.2,0.0026,7.52


In [ ]:
# Reporte de resultados: Costo y tiempo de procesamiento
print("Tiempo procesamiento promedio [s]: ", "{:.4f}".format(df_final_encs["proc_time"][df_final_encs["proc_time"].apply(type) != str].mean()))
print("Precio USD / comentario promedio: ", "{:.4f}".format(df_final_encs["cost"][df_final_encs["cost"].apply(type) != str].mean()))

# Show dataframe
# pd.set_option('display.max_colwidth', None)
# display(df_final.sample(n=2))
# pd.set_option('display.max_colwidth', 25)

Tiempo procesamiento promedio [s]:  15.7404
Precio USD / comentario promedio:  0.0027


##### **Guardar resultados**

In [ ]:
df_final_encs['categoria_email'] = categorias.apply(lambda x: x[x == x.max()].index[0], axis=1)
df_final_encs.reset_index(inplace=True, drop=True)


NameError: ignored

In [ ]:
path_resultados = 'data/results'
results_filename = 'U.csv' # ponga aquí el nombre del archivo de resultados que va a generar

df_final_encs.to_csv(os.path.join(path_resultados, results_filename), index=False)

### 📧 **Emails**

In [ ]:
from pathlib import Path
import pandas as pd
from gpt4service import utils

# Ruta al directorio que contiene los archivos .eml
path = Path('data/mails')
# Obtener la lista de archivos .eml en el directorio especificado
eml_files = list(path.glob('*.eml'))

file_names = []
file_paths = []
contents = []

# Iterar sobre cada archivo .eml
for file in eml_files:
    # Obtener el nombre de archivo y el contenido del correo electrónico utilizando la función get_content_email
    filename, email_content = utils.get_content_email(file)
    # Agregar el nombre de archivo, ruta de archivo y contenido a las listas correspondientes
    file_names.append(filename)
    contents.append(email_content)
    file_paths.append(file)

# Crear un DataFrame a partir de las listas de datos
df_eml = pd.DataFrame([file_names, file_paths, contents]).T
df_eml.columns = ['file_name', 'file_paths', 'content']

df_eml.head()

##### **Single**

In [ ]:
servicer.gpt_email_query(r'data/mails/Fwd_ ACLARACIрN DESCRIPCIрN _AJUSTE PAGO_.eml')

##### **Batch emails**

In [ ]:
df_loop = df_eml[3:5].copy() # use para procesar parte del df
# df_loop = df_eml.copy() # use para procesar todo el df
# use para procesar un dataframe con la información de los emls
df_final_emls = servicer.get_emails_df(df_loop)
df_final_emls

In [ ]:
# Reporte de resultados: Costo y tiempo de procesamiento
print("Tiempo procesamiento promedio [s]: ", "{:.4f}".format(df_final_emls["proc_time"][df_final_emls["proc_time"].apply(type) != str].mean()))
print("Precio USD promedio: ", "{:.4f}".format(df_final_emls["cost"][df_final_emls["cost"].apply(type) != str].mean()))

##### **Guardar resultados**

In [ ]:
path_resultados = 'data/results'
results_filename = 'correos_prueba.csv' # ponga aquí el nombre del archivo de resultados que va a generar
# guardar df sin el contenido de los emls
df_final_emls.drop(columns=['content']).to_csv(os.path.join(path_resultados, results_filename), index=False)